# Wirtualne terrarium

### Joanna Tustanowska
### 27.11.2018, Python4Beginners

**Plan na dziś**

**1. Jak zarządzać swoją instalacją Pythona i nie zwariować**
    * dlaczego powinniśmy to robić
    * jakie mamy narzędzia i jak działają
    * dobre praktyki
    
**2. Biblioteka requests - HTTP i API w Pythonie**
    * protokół HTTP (co to jest protokół i co to jest HTTP)
    * JSON API
    * jak wypełnić formularz w pythonie?
    * co robić, gdy potrzebne jest zalogowanie się?
    * do poczytania (scrapy, httpie/postman/curl, więcej o API design)
    


# Część 1: 
# Jak zarządzać swoją instalacją Pythona i nie zwariować


## Wstęp - gdzie mieszka Python?

Przykład - domyślna instalacja Pythona na Linuksie. To tylko niektóre pliki.

```console
$ which python
/bin/python
```

```console
$ ls /usr/lib/python3.7
-rw-r--r--  1 root root 5,5K 10-22 12:41 abc.py
-rwxr-xr-x  1 root root  20K 10-22 12:41 base64.py
...
drwxr-xr-x 88 root root 4,0K 11-16 15:26 site-packages
...
```

* `site-packages` to katalog, do którego trafiają zewnętrzne pakiety Pythona


Windows & Mac

## `pip` - narzędzie do instalacji pakietów

`pip` - narzędzie dystrybuowane razem z Pythonem (od wersji 3.4)

`$ pip install [nazwa pakietu]`  / można też: `$ python -m pip install [nazwa pakietu]`

np. `pip install maya>0.4.3` 

Zainstaluje pakiet w podanej wersji (lub najnowszy) oraz jego zależności.

Instalacja do `site-packages` użytkownika (nie wymaga uprawnień administratora):

`$ pip install --user [nazwa paczki]`



### Odinstalowanie pakietu

`$ pip uninstall [nazwa pakietu]`

Odinstaluje pakiet, pozostawi jego zależności.


### Lista zainstalowanych pakietów

```console
$ pip freeze
dateparser==0.7.0
humanize==0.5.1
pendulum==1.5.1
python-dateutil==2.7.5
[...]
```

`pip freeze > requirements.txt`

### Instalacja z requirements.txt

`$ pip install -r requirements.txt`


### Wyszukiwanie pakietów w (domyślnie w PyPI https://pypi.org/)

`$ pip search [nazwa pakietu]`

### Inne cechy

* `pip` zapisuje pobrane pakiety w cache'u, dla szybszej ponownej instalacji
* `pip check` - sprawdzi, czy w środowisku zainstalowane są wszystkie wymagane zależności, oraz czy nie ma konfliktów (np. pakiet A wymaga pakietu C w wersji <1.0, zaś pakiet B wymaga pakietu C w wersji >1.0)
* Dokumentacja `pip` do poczytania: https://pip.pypa.io/en/stable/user_guide/


## Nadchodzą problemy...

* W środowisku Pythona możemy mieć tylko jedną wersję danego pakietu, co może rodzić konflikty zależności:
    * np. pracujemy nad swoim projektem wykorzystującym najnowszą wersję pakietu A
    * i jednocześnie chcemy uruchomić projekt, w którym wymagana jest starsza wersja pakietu A.
* Na Linuksie i OSX Python jest używany przez system operacyjny. Pakiety (lub ich wersje), które doinstalujemy w głównym katalogu mogą zdestabilizować narzędzia systemowe...
* ...i odwrotnie, przy aktualizacji system może nam nadpisać wersje pakietów
* Możemy mieć problem z uporządkowaniem wymagań naszego projektu -- a chcielibyśmy móc uruchomić go nie tylko na naszym komputerze.

### Pytania do publiczności:
    
* (linux) czy zawsze potrzebujemy praw administratora (roota), żeby doinstalować pakiety do systemowego Pythona?
* (all) czy posprzątam swoje środowisko, jeśli odpalę `pip uninstall` dla każdego pakietu, który zainstalowałam przez `pip install`?
* (linux) czy mogę coś zepsuć wywołując `sudo pip install [paczka]`?
* (all) jak zainstalować pakiet A w wersji NIE NOWSZEJ niż 2.1?

## Środowisko wirtualne / virtual environment

* Kopia środowiska Pythona, tworzona na żądanie
   * binarka `python`
   * biblioteka standardowa
   * `pip` + skrypty do aktywacji i dezaktywacji środowiska
   * instalujemy tylko pakiety, których potrzebujemy w konkretnym projekcie

* Izolacja od pozostałych środowisk :+1:

### Sposób użycia - moduł `venv`

```console
$ python -m venv some/path/to/some_project_venv
```

Utworzy środowisko w zadanym katalogu.

```console
$ ls some_project_venv
bin  include  lib  lib64

$ ls -l some_project_venv/bin
-rw-r--r-- 1 asia asia 2248 11-25 22:05 activate
-rwxr-xr-x 1 asia asia  257 11-25 22:05 pip
lrwxrwxrwx 1 asia asia   11 11-25 22:05 python -> /bin/python
[...]

$ ls some_project_venv/lib/Python3.7
site-packages
```

### Aktywacja środowiska - skrypt shellowy `activate`


Platform  	|Shell 	|Command to activate virtual environment
---------------|----|-----------------------------------------
Posix 	|bash/zsh 	|`$ source <venv>/bin/activate`
  |  	fish 	|`$ . <venv>/bin/activate.fish`
  |  	csh/tcsh 	|`$ source <venv>/bin/activate.csh`
Windows 	|cmd.exe 	|`C:\> <venv>\Scripts\activate.bat`
  |  	PowerShell 	|`PS C:\> <venv>\Scripts\Activate.ps1`

https://docs.python.org/3/library/venv.html#creating-virtual-environments

### Jak to działa?

Zmiany w bieżącej sesji terminala:

* modyfikacja *zmiennej środowiskowej* `PATH` - Skierowanie na `pythona`, `pip`a i inne pliki wykonywalne pod nowym adresem
* ustawienie zmiennej środowiskowej `VIRTUAL_ENV` - po uruchomieniu Python wie, skąd ładować importowane pakiety, a pip wie, gdzie instalować
* stworzenie komendy (funkcji w shellu) `deactivate`


### Zmienne środowiskowe

Różne nazwane wartości, z których korzystają procesy uruchamiane w systemie operacyjnym. Gdy używamy terminala (shella=powłoki), możemy zmodyfikować zmienne środowiskowe tak, by wpłynąć na działanie programu, który za chwilę w tym shellu uruchomimy. Ze zmiennych środowiskowych korzysta zarówno Windows, Linuksy, jak i OSX.

Przykłady:

* `HOME` - adres katalogu domowego bieżącego użytkownika
* `USER` - nazwa bieżącego użytkownika
* `PATH` - lista katalogów, w których shell będzie szukał pliku wykonywalnego, gdy wpiszemy jakąś komendę bez podania pełnej ścieżki. Np. `python` :-)


### Przykład

```bash
# przed aktywacją virtual enva
$ echo $PATH
/home/asia/.local/bin:/home/asia/bin:/usr/local/bin:/usr/bin:/bin
$ which python
/bin/python
$ source some_project_venv/bin/activate
```

### Przykład c.d.

```bash
# po aktywacji
(some_project_venv) $ echo $PATH
/home/asia/some_project_venv/bin:/home/asia/.local/bin:/home/asia/bin:/usr/local/bin:/usr/bin:/bin
(some_project_venv) $ which python
/home/asia/some_project_venv/bin/python
(some_project_venv) $ echo $VIRTUAL_ENV
/home/asia/some_project_venv

(some_project_venv) $ deactivate
$ which python
/bin/python
```


## Jak tego używać? Przykład 1

Środowisko wirtualne w katalogu projektu:

* w katalogu projektu (powiedzmy `my_project`) odpalam `python -m venv env_my_project`
* [jeśli używam `git`a, wpisuję katalog `env_my_project` do pliku `.gitignore`]
* aktywuję środowisko `source env_my_project/bin/activate` (za każdym razem, gdy w nowym terminalu uruchamiam pliki projektu)
* instaluję potrzebne pakiety (to tylko raz)
* gdy projekt działa, zapisuję jego wymagania w pliku requirements: `pip freeze > requirements.txt`. Przyda się, gdy będę budować środowisko od nowa (`pip install -r requirements.txt`)
* gdy kończę pracę nad projektem i nie chcę zamykać okna terminala, dezaktywuję środowisko poleceniem `deactivate`
   

## Dobre rady

* NIE instalujemy pakietów w głównym środowisku Pythona. To się może źle skończyć.
* narzędzia, do których chcemy mieć dostęp w całym systemie (jako nasz użytkownik) instalujemy przez `pip install --user [paczka]`
* do każdego projektu w pythonie używamy osobnego virtualenva

### Pytania do publiczności

* czy mogę stworzyć środowisko wirtualne z wersją Pythona, której nie mam zainstalowanej w systemie?
* czy jeśli zainstaluję jakiś pakiet w wirtualnym środowisku, to będzie on dostępny dla głównego, systemowego Pythona?
* gdzie będzie 'widoczny' mój virtualenv, gdy go aktywuję?
* co się stanie, gdy w terminalu skasuję zawartość zmiennej środowiskowej `PATH`, np. poleceniem `export PATH=""`?

## Jak tego używać?  :-(

* :-( muszę ręcznie stworzyć i nazwać środowisko wirtualne
* :-( muszę pamiętać, jak mój virtualenv się nazywa i gdzie w systemie plików się znajduje
* :-( aby aktywować środowisko muszę wpisywać długie polecenia
* :-( moje środowisko wirtualne ma masę niepotrzebnych pakietów i nie jestem pewna/pewien, które dokładnie pakiety mogę usunąć

### Trochę prościej - `virtualenvwrapper`

https://virtualenvwrapper.readthedocs.io/en/latest/

* środowiska wirtualne w jednym miejscu
* wygodne polecenia, np. `mkvirtualenv [nazwa]`, `workon [nazwa]`
* możliwość skonfigurowania auto-aktywacji środowiska po wejściu do katalogu projektu


## `pipenv` - poziom wyżej

https://pipenv.readthedocs.io/en/latest/

Rekomendowany system zarządzania zależnościami w projektach pythonowych.

Używa `pip` i `venv` za nas, dodając do tego wiele mądrych automatyzacji.

### `pipenv` - szybki start

https://pipenv.readthedocs.io/en/latest/install/#installing-pipenv
    
OSX: `brew install pipenv`

Win/Linux: `pip install --user pipenv`

### `pipenv` - użycie w nowym projekcie

* przechodzimy w konsoli do katalogu projektu: `cd my_project`
* od razu coś instalujemy, np. bibliotekę `requests`: `pipenv install requests`
* `pipenv` automatycznie znajduje/tworzy środowisko wirtualne (w swoim centralnym katalogu)
* automatycznie tworzy w projekcie pliki `Pipfile` i `Pipfile.lock` (lepszy odpowiednik `requirements.txt`)
* w katalogu, w którym znajduje się Pipfile, aby stworzyć środowisko wystarczy wywołać samo `pipenv install`


```console
➜  experiments$ mkdir my_project
➜  experiments$ cd my_project 
➜  my_project$ pipenv install requests
Creating a virtualenv for this project…
Pipfile: /home/asia/experiments/my_project/Pipfile
Using /usr/bin/python (3.7.1) to create virtualenv…
✔ Complete 
Using interpreter /bin/python
New python executable in /home/asia/venvs/my_project-F8Fo0uYg/bin/python
Installing setuptools, pip, wheel...done.
Virtualenv location: /home/asia/venvs/my_project-F8Fo0uYg
Creating a Pipfile for this project…
Installing requests…
Adding requests to Pipfile's [packages]…
✔ Installation Succeeded 
Pipfile.lock not found, creating…
Locking [dev-packages] dependencies…
Locking [packages] dependencies…
✔ Success! 
Updated Pipfile.lock (444a6d)!
Installing dependencies from Pipfile.lock (444a6d)…
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 5/5 — 00:00:01
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.

```

### `pipenv` - aktywacja środowiska

* aktywujemy środowisko poleceniem `pipenv shell`, deaktywujemy poleceniem `exit` (uruchamiamy/kończymy nowy shell)
* możemy też uruchomić swoje programy bez wcześniejszej aktywacji, np. `pipenv run python my_program.py`

### `pipenv` - Pipfile + Pipfile.lock

* `Pipfile` - deklarowane zależności projektu (np. `requests`) - czytelny dla człowieka
* każda instalacja/deinstalacja aktualizuje `Pipfile`
* `Pipfile.lock` - wszystkie zależności (czyli np. także zależności biblioteki `requests`), stan całego środowiska - czytelny dla pipenva

### Prosty `Pipfile`

```toml
[[source]]
name = "pypi"
url = "https://pypi.org/simple"
verify_ssl = true

[dev-packages]

[packages]
requests = "*"

[requires]
python_version = "3.7"
```

### `pipenv` - Sprzątanie środowiska

`$ pipenv uninstall [paczka]` - usuwa pakiet, aktualizuje Pipfile


`$ pipenv clean` - usunie niepotrzebne zależności pakietów, których już nie ma w Pipfile


### Pytania do publiczności

* jak się ma `pip` + `venv` do `pipenv`?
* jakich narzędzi będę używać do zarządzania zależnościami w swoim projekcie i czemu to będzie `pipenv`?

## Ile Pythonów zmieści się w jednym systemie?


Wiele. Możemy mieć zainstalowanych kilka wersji Pythona (np. 2.7.x, 3.6.x oraz 3.7.x), a nawet kilka dystrybucji tej samej wersji (Anaconda oraz oficjalny Python https://python.org ).



Zarządzanie wersjami Pythona - do poczytania https://github.com/pyenv/pyenv

## Podsumowanie

* po co nam środowiska wirtualne?
   * izolacja projektów 
       * różne wersje zależności
       * niemodyfikowanie Pythona systemowego
   * powtarzalność instalacji
   * przewidywalność działania oprogamowania - pakiety mogą wpływać na działanie innych pakietów
   
* jakie mamy narzędzia?
   * wbudowane w Pythona - `pip` i `venv`
   * ładne opakowania do powyższych - `virtualenvwrapper`, a przede wszystkim `pipenv`
   
* dobre praktyki:
   * używać środowisk wirtualnych, **zawsze**!

Koniec części 1 :)